In [1]:
from google import adk
from google.adk.agents import LlmAgent
from vertexai.preview.reasoning_engines import AdkApp
from typing import List, Dict, Any
from google.adk.tools import ToolContext
from google.adk.sessions import InMemorySessionService
from vertexai import agent_engines
from google.adk.tools import ToolContext
import vertexai

import os
os.sys.path.append("/home/jupyter/code_test/adk_rag/src/")
from common.vector_store import VectorStore
from common.embedding_generator import EmbeddingGenerator

In [3]:
vs = VectorStore()
eg = EmbeddingGenerator()

In [51]:
def retrieve_answer(query: str):
    #query_embedding = eg.generate_single_embedding(query)
    #return vs.search_vectors(query_embedding)
    return "yeah"

In [52]:
rag_agent = LlmAgent(
    name="rag_agent",
    model="gemini-2.0-flash",
    description="An Agent that can provide answer to user questions based on retrieved context",
    instruction=(
        "You are an question answering agent who will answer user's question by retrieving knowledge from the knowledge base by running 'retrieve_answer(query)' function"
    ),
    tools=[retrieve_answer],
)


In [41]:
vertexai.init(staging_bucket="gs://yuan_evernote_rag_cs")

In [48]:
!pwd

/home/jupyter/code_test/adk_rag/src/agent


In [54]:
#remote_app.delete(force=True)

NameError: name 'remote_app' is not defined

In [53]:

remote_app = agent_engines.create(
    agent_engine=rag_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]==1.79.0"   
    ],
    #extra_packages=["/home/jupyter/code_test/adk_rag/src/common"]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'pydantic': '2.11.4', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.92.0'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are incompatible: {'google-cloud-aiplatform==1.92.0 (required: ==1.79.0)'}
The following requirements are appended: {'pydantic==2.11.4', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]==1.79.0', 'pydantic==2.11.4', 'cloudpickle==3.1.1']
Using bucket yuan_evernote_rag_cs
Wrote to gs://yuan_evernote_rag_cs/agent_engine/agent_engine.pkl
Writing to gs://yuan_evernote_rag_cs/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://yuan_evernote_rag_cs/agent_engine/dependencies.tar.gz
failed to generate schema for stream_query: `stream_query` is not fully defined; you should define `ContentDict`, then call `stream_query.model_rebuild()`.

For

KeyboardInterrupt: 

In [32]:

# Wrap the agent in an AdkApp for deployment
app = AdkApp(agent=rag_agent)

In [33]:
session = app.create_session(user_id="u_123")
session

Session(id='34fd7f4f-11b8-4a16-b91d-4267edc934e7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1747407738.406505)

In [36]:
events = []
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="what is LY's email",
):
    events.append(event)

Exception in thread Thread-24 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/opt/conda/envs/myenv/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/myenv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/envs/myenv/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/myenv/lib/python3.10/site-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/opt/conda/envs/myenv/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/opt/conda/envs/myenv/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/opt/conda/envs/myenv/lib/python3.10/site-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in

In [37]:
events

[{'content': {'parts': [{'function_call': {'id': 'adk-e6830db1-58a7-49d2-9f94-2b15a13fb052',
      'args': {'query': "What is LY's email address?"},
      'name': 'retrieve_answer'}}],
   'role': 'model'},
  'invocation_id': 'e-b5bd3971-29cf-4a78-b77e-3ce9a4213f7f',
  'author': 'rag_agent',
  'actions': {'state_delta': {},
   'artifact_delta': {},
   'requested_auth_configs': {}},
  'long_running_tool_ids': set(),
  'id': 'pH1dBQPQ',
  'timestamp': 1747407752.809819}]